In [0]:
!pip install pulp -q

SQ@20250 で利益最大ポジ

制約条件
*  初期コスト（１０日清算値時点）125以下
*  売り枚数１０枚以下
*  18000~23000で最大支払い額250


In [28]:
import pulp as pu
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/zaq9/case/master/case20190110/data20190110.csv',index_col=0)

v_sq = 20375

class Payoff():
    def __init__(self, name):
        self.type = name[0]
        self.strike = int(name[1:])
    def __call__(self, u):
        v = u - self.strike
        if(self.type =='C'): 
            return v if v>0 else 0
        else:
            return -v if v<0 else 0

          
          
df["buy"] = df.index.map(lambda x: pu.LpVariable(x, 0, 20, pu.LpInteger))
df["sell"] = df.index.map(lambda x: pu.LpVariable(x+'_s', 0, 10, pu.LpInteger))
df["payoff"] = df.index.map(lambda x: Payoff(x))


def pay(s, objs): return pu.lpSum(pu.lpDot(objs, [p(s) for p in df.payoff]))
def v(s):   return pay(s, df['buy']) - pay(s, df['sell'])

cost = pu.lpSum(pu.lpDot(df['buy'], df['PRICE'])) -pu.lpSum(pu.lpDot(df['sell'], df['PRICE']))
m = pu.LpProblem('Max', sense=pu.LpMaximize)
m += v(v_sq ) -cost
m += cost <= 125
m += pu.lpSum(df['sell']) <= 10  # 合計売り枚数

for sq in range(18000,23000,125): m += v(sq) >= -250 
    
m.solve()
df['ans'] = df.buy.map(lambda x: x.value()) - df.sell.map(lambda x: x.value())
print(f"最大収益 ={m.objective.value():.0f}(@SQ={v_sq}):コスト={sum(df.ans * df.PRICE):.0f}")

for x in df.query( ' ans != 0 ').itertuples():
    print(f"{x.Index}[{x.ans:.0f}]@{x.PRICE} >> {x.payoff(v_sq)}")
 

最大収益 =1035(@SQ=20375):コスト=90
C20250[9]@48 >> 125
C20375[-9]@19 >> 0
P20125[1]@94 >> 0
P20375[-1]@265 >> 0
